# <center> Introduction to Hadoop MapReduce </center>

Python Jupyter notebook supports execution of Linux command inside the notebook cells. This is done by adding the **!** to the beginning of the command line. It should be noted that each command begins with a **!** will create a new bash shell and close this cell once the execution is done:
- Full path is required
- Temporary results and environmental variables will be lost

We need to initialize Kerberos authentication mechanism

Interaction with Hadoop Distributed File System is done through `hdfs` and its sub-commands

In [1]:
!hdfs

Usage: hdfs [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]

  OPTIONS is none or any of:

--buildpaths                       attempt to add class files from build tree
--config dir                       Hadoop config directory
--daemon (start|status|stop)       operate on a daemon
--debug                            turn on shell script debug mode
--help                             usage information
--hostnames list[,of,host,names]   hosts to use in worker mode
--hosts filename                   list of hosts to use in worker mode
--loglevel level                   set the log4j level for this command
--workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

cacheadmin           configure the HDFS cache
crypto               configure HDFS encryption zones
debug                run a Debug Admin to execute HDFS debug commands
dfsadmin             run a DFS admin client
dfsrouteradmin       manage Router-based federation
ec                   run a HD

In [2]:
!hdfs dfs

Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] [-t <thread count>] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] [-e] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-v] [-x] <path> ...]
	[-expunge [-immediate]]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] [-skip-empty-file] <src> <localdst>]
	[-head <file>]
	[-help [cmd ...]]
	[-ls [-C] [-d] [-h] [-q] [-R] [-t] [-

In [3]:
!hdfs dfs -ls /

Found 4 items
drwxr-xr-x   - nurcahyopujo hadoop          0 2021-04-26 23:27 /data_training
drwxr-xr-x   - root         hadoop          0 2021-04-26 13:15 /hbase
drwxrwxrwt   - hdfs         hadoop          0 2021-04-24 06:27 /tmp
drwxrwxrwt   - hdfs         hadoop          0 2021-04-23 23:21 /user


### Challenge

Create a directory named **intro-to-hadoop** inside your user directory on HDFS

In [9]:
!hdfs dfs -mkdir -p /user/nurcahyo/intro-to-hadoop

In [10]:
!hdfs dfs -ls /user/nurcahyo

Found 1 items
drwxr-xr-x   - root hadoop          0 2021-04-26 23:34 /user/nurcahyo/intro-to-hadoop


### Challenge

Upload the **text** directory into the newly created **intro-to-hadoop** directory. 

In [ ]:
!hdfs dfs -put

In [ ]:
!hdfs dfs -put text intro-to-hadoop

### Challenge 

Check the health status of the directories above in HDFS using fsck:
```
hdfs fsck <path-to-directory> -files -blocks -locations
```

In [ ]:
!hdfs --config ~/hadoop_palmetto/config fsck intro-to-hadoop/text/complete-shakespeare.txt -files -blocks -locations

## MapReduce Programming Paradigm

**What is “map”?**
– A function/procedure that is applied to every individual
elements of a collection/list/array/…

```
int square(x) { return x*x;}
map square [1,2,3,4] -> [1,4,9,16]
```

**What is “reduce”?**
– A function/procedure that performs an operation on a list.
This operation will “fold/reduce” this list into a single value
(or a smaller subset)

```
reduce ([1,2,3,4]) using sum -> 10
reduce ([1,2,3,4]) using multiply -> 24
```

MapReduce is an old concept in functional programming. It is naturally applicable in HDFS: 
- `map` tasks are performed on top of individual data blocks (mainly to filter and decrease raw data contents while increase data value
- `reduce` tasks are performed on intermediate results from `map` tasks (should now be significantly decreased in size) to calculate the final results. 

## 1. The Hello World of Hadoop: Word Count

In [ ]:
!mkdir codes

In [ ]:
!hdfs dfs -cat intro-to-hadoop/text/complete-shakespeare.txt \
    2>/dev/null | head -n 100

In [ ]:
%%writefile codes/wordcountMapper.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python                                          
import sys                                                                                                
for oneLine in sys.stdin:
    oneLine = oneLine.strip()
    for word in oneLine.split(" "):
        if word != "":
            print ('%s\t%s' % (word, 1)) 

In [ ]:
!hdfs dfs -cat intro-to-hadoop/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py

In [ ]:
!hdfs dfs -cat intro-to-hadoop/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py \
    | sort

In [ ]:
%%writefile codes/wordcountReducer.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python
import sys

current_word = None
total_word_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split("\t", 1)
    try:
        count = int(count)
    except ValueError:
        continue
    
    if current_word == word:
        total_word_count += count
    else:
        if current_word:
            print ("%s\t%s" % (current_word, total_word_count))
        current_word = word
        total_word_count = 1
        
if current_word == word:
    print ("%s\t%s" % (current_word, total_word_count))

In [ ]:
!hdfs dfs -cat intro-to-hadoop/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py \
    | sort \
    | python ./codes/wordcountReducer.py

In [ ]:
!hdfs dfs -rm -R intro-to-hadoop/output-wordcount
!mapred streaming \
    -input intro-to-hadoop/text/complete-shakespeare.txt \
    -output intro-to-hadoop/output-wordcount \
    -file ./codes/wordcountMapper.py \
    -mapper wordcountMapper.py \
    -file ./codes/wordcountReducer.py \
    -reducer wordcountReducer.py

In [ ]:
!hdfs dfs -ls intro-to-hadoop/output-wordcount

In [ ]:
!hdfs dfs -cat intro-to-hadoop/output-wordcount/part-00000 \
    2>/dev/null | head -n 100

### Challenge

Modify *wordcountMapper.py* so that punctuations and capitalization are no longer factors in determining unique words

In [ ]:
%%writefile codes/wordcountEnhancedMapper.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python                                          
import sys                     
import string

translator = str.maketrans('', '', string.punctuation)

for oneLine in sys.stdin:
    oneLine = oneLine.strip()
    for word in oneLine.split(" "):
        if word != "":
            newWord = word.translate(translator).lower()
            print ('%s\t%s' % (_______, 1)) 

In [ ]:
!hdfs dfs -rm -R intro-to-hadoop/output-wordcount-enhanced
!ssh dsciutil yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input intro-to-hadoop/text/gutenberg-shakespeare.txt \
    -output intro-to-hadoop/output-wordcount \
    -file ____________________________________________________ \
    -mapper _____________________ \
    -file ____________________________________________________ \
    -reducer _____________________ \